In [1]:
import asyncio
import nest_asyncio
import motor.motor_asyncio
from datetime import date, datetime, timedelta
from collections import defaultdict
import pprint
import sys
import pandas as pd
import numpy as np
import ta
import os
import matplotlib.pyplot as plt
import talib
from ipywidgets import IntProgress
from IPython.display import display

nest_asyncio.apply()
%matplotlib inline

sys.path.insert(0, os.path.abspath('../'))
import abupy
from abupy.UtilBu.ABuDateUtil import str_to_datetime

NumExpr defaulting to 8 threads.


In [2]:
client = motor.motor_asyncio.AsyncIOMotorClient('mongodb://localhost:27017')
db = client['symbol-db']
loop = asyncio.get_event_loop()

In [3]:
# spider cmd
#%scrapy crawl StockHisTraderSpider --logfile test.log
# symbol(2474) 可成
yy, mm, dd = date.today().isoformat().split("-")[0:3]
cdate = "{0}-{1}-{2}".format(yy, mm, dd)
#cdate = '2021-05-21'
today = str_to_datetime(cdate)
symbol = '2474'
trader = u'合庫證券'

C_MIN, C_MAX = 50, 1000 #close Min:Max
V_MAX = 500 # volume >= V_max
broker_sampleD = 1 # broker sample [n]Days
candidates = []

S_endDate = today
S_startDate = S_endDate - timedelta(days=70)
B_endDate = today
B_startDate = B_endDate - timedelta(days=15)
M_endDate = today
M_startDate = M_endDate - timedelta(days=70)
I_endDate = today
I_startDate = I_endDate - timedelta(days=70)
Side = 'Call' #'Call/Put'

In [4]:
# filter rank[:10], 全市場,買家對個股成交比重 
async def do_find_max_calls_weight_symbol():
    # test find max count of call symbols at sample time window
    pipeline = [
        {"$match": { "date": { "$lte": B_endDate} } },
        {"$match": { "date": { "$gte": B_startDate} } },
        {"$unwind": "$traders" },
        
        {"$group": { "_id": { "tdname": "$traders.tdname", "symbol": "$symbol"}, 
                    "sum_puts":  { "$sum": "$traders.put" }, 
                    "sum_calls": { "$sum": "$traders.call" },
                    "sum_weights": { "$sum": "$traders.weight" },
                    "max_weight": { "$max": "$traders.weight" },
                    "sample_days": { "$sum": 1}
                   }  },
        
        {"$project": {"_id": 0,
                      "avg_weights": {"$round" : [{"$divide": ["$sum_weights", "$sample_days"]}, 1]}, 
                      "max_weight": "$max_weight",
                      "sample_days": "$sample_days",
                      "diff_calls_puts": { "$cond": [ { "$eq": [ "$sum_puts", 0 ] }, "$sum_calls", {"$round" : [{"$divide": ["$sum_calls", "$sum_puts"]}, 1]}]}, 
                      "sum_puts": "$sum_puts", "sum_calls": "$sum_calls",
                      "sum_all": {"$add": ["$sum_puts", "$sum_calls"]},
                      "symbol": "$_id.symbol", "tdname": "$_id.tdname"
                     } },
        
         #{"$match":{ "$expr": {"$and": [
         #               { "$gte": ["$diff_calls_puts", 100 ] },
         #               { "$gte": ["$avg_weights", 10 ] }
         #            ]
         #           }}},
             
        {"$sort": { "diff_calls_puts": -1, "max_weight": -1, "sample_days": -1,  "sum_calls": -1, "sum_puts": 1 } },
        #{"$limit": 100 }
        ]
    collect = db['StockHisTrader'].aggregate(pipeline, allowDiskUse=True)
    results = []
    async for cursor in collect: # clean cursor ptr and store to list
        results.append(cursor)
    df = pd.DataFrame(results)
    return df


In [5]:
df = loop.run_until_complete(do_find_max_calls_weight_symbol())     
df = df[df['symbol'] == '2474']
df[:30]

,avg_weights,max_weight,sample_days,diff_calls_puts,sum_puts,sum_calls,sum_all,symbol,tdname
1761,3.5,4.28,2,156.0,1.0,156.0,157.0,2474,兆豐-北高雄
2368,3.2,3.67,2,126.0,0.0,126.0,126.0,2474,元大-金華
4266,1.8,2.80,2,87.0,0.0,87.0,87.0,2474,元富-台南
4302,5.6,5.58,1,86.0,1.0,86.0,87.0,2474,合庫-台中
7048,4.2,4.15,1,56.0,0.0,56.0,56.0,2474,華南永昌-長虹
...,...,...,...,...,...,...,...,...,...
23524,1.4,1.41,1,20.0,0.0,20.0,20.0,2474,元大-基隆孝二
23528,1.4,1.41,1,20.0,1.0,20.0,21.0,2474,元大-復北
24305,0.8,0.82,1,20.0,0.0,20.0,20.0,2474,台灣企銀-太平
25239,0.7,0.72,1,19.0,0.0,19.0,19.0,2474,富邦-建國


In [6]:
# filter rank[:10], 全市場,賣家對個股成交比重 
async def do_find_max_puts_weight_symbol():
    # test find max count of put symbols at sample time window
    pipeline = [
        # add lookup hisstock filter min/max close and margin rate, total_puts/total_volume = total_weight
        {"$match": { "date": { "$lte": B_endDate } } },
        {"$match": { "date": { "$gte": B_startDate } } },
        {"$unwind": "$traders" },
        
        {"$group": { "_id": { "tdname": "$traders.tdname", "symbol": "$symbol"}, 
                    "sum_puts":  { "$sum": "$traders.put" }, 
                    "sum_calls": { "$sum": "$traders.call" },
                    "sum_weights": { "$sum": "$traders.weight" },
                    "max_weight":  { "$max": "$traders.weight" },
                    "sample_days": { "$sum": 1}
                   }  },
        
        {"$project": {"_id": 0,
                      "avg_weights": {"$round" : [{"$divide": ["$sum_weights", "$sample_days"]}, 1]},
                      "max_weight": "$max_weight",
                      "sample_days": "$sample_days",
                      "diff_puts_calls": {"$round" : [{"$subtract": ["$sum_puts", "$sum_calls"]}, 1]}, 
                      "sum_puts": "$sum_puts", "sum_calls": "$sum_calls",
                      "sum_all": { "$add": ["$sum_puts", "$sum_calls"]},
                      "symbol": "$_id.symbol", "tdname": "$_id.tdname"
                     } },
        
         {"$match":{ "$expr": {"$and": [
                        { "$gte": ["$diff_puts_calls", 100 ] },
                        { "$gte": ["$avg_weights", 10 ] }]
                    }}},
             
        {"$sort": { "diff_puts_calls": -1, "max_weight": -1,"sample_days": -1,  "sum_puts": -1, "sum_calls": 1 } },
        {"$limit": 20 }
        ]
    collect = db['StockHisTrader'].aggregate(pipeline, allowDiskUse=True)
    results = []
    async for cursor in collect: # clean cursor ptr and store to list
        results.append(cursor)
    df = pd.DataFrame(results)
    return df

In [7]:
df = loop.run_until_complete(do_find_max_puts_weight_symbol()) 
df[:3]

,avg_weights,max_weight,sample_days,diff_puts_calls,sum_puts,sum_calls,sum_all,symbol,tdname
0,20.2,47.80,3,48891.0,50051.0,1160.0,51211.0,2883,統一
1,19.0,38.12,9,43409.0,244314.0,200905.0,445219.0,00632R,元大證券
2,13.5,20.91,4,42582.0,43839.0,1257.0,45096.0,00893,元大證券


In [8]:
# filter rank[:10], 全市場,特定買賣家 對個股當沖隔日沖成交比重 
async def do_find_trader_with_all_symbols():
    # test find max count of put symbols at sample time window
    pipeline = [
        # add lookup hisstock filter min/max close and margin rate, total_puts/total_volume = total_weight
        {"$match": { "date": { "$lte": B_endDate } } },
        {"$match": { "date": { "$gte": B_startDate } } },
        {"$unwind": "$traders" },
        
        {"$group": { "_id": { "tdname": "$traders.tdname", "symbol": "$symbol"}, 
                    "sum_puts":  { "$sum": "$traders.put" }, 
                    "sum_calls": { "$sum": "$traders.call" },
                    "sum_weights": { "$sum": "$traders.weight" },
                    "max_weight":  { "$max": "$traders.weight" },
                    "sample_days": { "$sum": 1 }
                   }  },
        
        {"$project": {"_id": 0,
                      "avg_weights": {"$round" : [{"$divide": ["$sum_weights", "$sample_days"]}, 1]}, 
                      "max_weight": "$max_weight",
                      "sample_days": "$sample_days",
                      "sum_puts": "$sum_puts", "sum_calls": "$sum_calls",
                      "sum_all": { "$add": ["$sum_puts", "$sum_calls"]},
                      "symbol": "$_id.symbol", "tdname": "$_id.tdname"
                     } },
        
         {"$match":{ "$expr": {"$and": [
                        { "$eq": ["$tdname", trader ] }
                     ]}
        }},
         
        # 隔日沖 sample day T ~ T[n+1]
        # TODO: 需加入 close * count 成交金額, 不然用 count filter 有可能找不到大戶籌碼
        # solution: limit 開大點
        {"$sort": { "sum_all": -1, "max_weight": -1, "avg_weights": -1, "sample_days": 1 } },
        {"$limit": 500 }
        ]
    collect = db['StockHisTrader'].aggregate(pipeline, allowDiskUse=True)
    results = []
    async for cursor in collect: # clean cursor ptr and store to list
        results.append(cursor)
    df = pd.DataFrame(results)
    return df

In [9]:
# 尋找 隔日沖分點明細
# 凱基-松山 2021/04/29 對聯發科(2454)當沖分析
#2021/05/03	342	1,395	1,737	-1,053
#2021/04/29	998	10	1,008	988

B_endDate = today
B_startDate = B_endDate - timedelta(days=70)
traders = [u'富邦-建國', u'凱基-松山', u'元大-土城永寧', u'國票-敦北法人', u'兆豐-中壢', u'兆豐-北高雄', u'統一-新竹', u'群益金鼎-大安']
trader = traders[1]
df = loop.run_until_complete(do_find_trader_with_all_symbols()) 
dft = df[df['symbol']=='2454']
dft

,avg_weights,max_weight,sample_days,sum_puts,sum_calls,sum_all,symbol,tdname
75,3.3,15.83,7,1496.0,1579.0,3075.0,2454,凱基-松山


In [10]:
async def do_find_symbol_correlation(symbol, S_endDate, S_startDate):
    pipeline = [
        {"$match":{ "$expr": {"$and": [
            { "$lte" : ["$date", S_endDate ]},
            { "$gte" : ["$date", S_startDate ]},
            { "$eq" : ["$symbol", symbol]}
            ]}
        }},
        {"$project": {"_id": 0 } } # no _id to next pipeline
    ]
    collect = db['StockHisData'].aggregate(pipeline, allowDiskUse=True)
    results = []
    async for cursor in collect: # clean cursor ptr and store to list
        results.append(cursor)
    df = pd.DataFrame(results)
    try:
        df = df.set_index('date').dropna().sort_index()
        df = df.drop(columns=['symbol'])
    except Exception as e:
        pass
    return df

In [11]:
async def do_find_trader_correlation(trader, symbol, B_endDate, B_startDate):    
    pipeline = [
        {"$match": { "$expr": {"$and": [
                        { "$lte": ["$date", B_endDate ] },
                        { "$gte": ["$date", B_startDate ] },
                        { "$eq": ["$symbol", symbol]}
                        ]}
                   }},

        { "$addFields": {
            "ctrader": 
                { "$filter": {
                    "input": "$traders",
                    "as": "ttrader",
                    "cond": { "$eq" : ["$$ttrader.tdname", trader]}
                }}
        }},
        
        { "$addFields": {
            "put": 
                { "$map": {
                    "input": "$ctrader",
                    "as": "ttrader",
                    "in": "$$ttrader.put"
                }},
            "call": 
                { "$map": {
                    "input": "$ctrader",
                    "as": "ttrader",
                    "in": "$$ttrader.call"
                }},
            "weight": 
                { "$map": {
                    "input": "$ctrader",
                    "as": "ttrader",
                    "in": "$$ttrader.weight"
                }},
            
        }},
        
        { "$project": { 
            "_id": 0,
            "put": { "$cond": [{"$size": "$put" }, { "$arrayElemAt": [ "$put", -1 ] }, 0 ] },
            "call": { "$cond": [{"$size": "$put" }, { "$arrayElemAt": [ "$call", -1 ] }, 0 ] },
            "weight": { "$cond": [{"$size": "$weight" }, { "$arrayElemAt": [ "$weight", -1 ] }, 0 ] },
            "date": "$date"
        }}
    ]
    
    collect = db['StockHisTrader'].aggregate(pipeline,allowDiskUse=True)
    results = []
    async for cursor in collect: # clean cursor ptr and store to list
        results.append(cursor)
    df = pd.DataFrame(results)
    try:
        df = df.set_index('date').dropna().sort_index()
    except Exception as e:
        pass
    return df

In [12]:
async def do_find_margin_correlation(symbol, M_endDate, M_startDate):
    pipeline = [
        {"$match":{ "$expr": {"$and": [
            { "$lte" : ["$date", M_endDate ]},
            { "$gte" : ["$date", M_startDate ]},
            { "$eq" : ["$symbol", symbol]}
            ]}
        }},
        {"$project": {"_id": 0,
                "MCall": "$MagPurchBalance",
                "SPut": "$ShortSaleBalance",
                "SDivM": "$ShortDivMagRatio",
                "date": "$date"
        } } # no _id to next pipeline
    ]
    collect = db['StockHisMargin'].aggregate(pipeline, allowDiskUse=True)
    results = []
    async for cursor in collect: # clean cursor ptr and store to list
        results.append(cursor)
    df = pd.DataFrame(results)
    try:
        df = df.set_index('date').dropna().sort_index()
    except Exception as e:
        pass
    return df

In [13]:
async def do_find_Institution_correlation(symbol, I_endDate, I_startDate):
    pipeline = [
        {"$match":{ "$expr": {"$and": [
            { "$lte" : ["$date", I_endDate ]},
            { "$gte" : ["$date", I_startDate ]},
            { "$eq" : ["$symbol", symbol]}
            ]}
        }},
        {"$project": {"_id": 0,
                "ForV": "$ForeignInvestor",
                "InvV": "$InvestmentTrust",
                "DelV": "$DealerSelf",
                "date": "$date"
        } } # no _id to next pipeline
    ]
    collect = db['StockInstitutionInvest'].aggregate(pipeline, allowDiskUse=True)
    results = []
    async for cursor in collect: # clean cursor ptr and store to list
        results.append(cursor)
    df = pd.DataFrame(results)
    try:
        df = df.set_index('date').dropna().sort_index()
    except Exception as e:
        pass
    return df

In [14]:
import pandas as pd
from ta.utils import dropna
from ta.volatility import BollingerBands

# 找出 broker vs symbol 全市場資料
symbols = df['symbol'].tolist()
symbol = symbols[0]
trader = df['tdname'].tolist()[0]

B_endDate = today
B_startDate = B_endDate - timedelta(days=70)
S_endDate = today
S_startDate = S_endDate - timedelta(days=70)
M_endDate = today
M_startDate = M_endDate - timedelta(days=70)
I_endDate = today
I_startDate = I_endDate - timedelta(days=70)


In [15]:
import pandas as pd
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.trend import EMAIndicator

def ask_color(s, column=['put', 'call']):
    is_up = pd.Series(data=False, index=s.index)
    is_up['up'] = s.loc[column[0]] < s.loc[column[1]]
    return ['background-color: red' if is_up.any() else '' for v in is_up]

def wrap_ta_bband(df): 
    indicator_bb = BollingerBands(close=df["C"], window=20, window_dev=2)
    # Add Bollinger Bands features
    df['bb_bbm'] = indicator_bb.bollinger_mavg()
    df['bb_bbh'] = indicator_bb.bollinger_hband()
    df['bb_bbl'] = indicator_bb.bollinger_lband()
    return df

def wrap_ta_ema(df):
    ma_5 = EMAIndicator(close=df["C"], window=5)
    ma_20= EMAIndicator(close=df["C"], window=20)
    ma_60= EMAIndicator(close=df["C"], window=60)
    df['ma_5']  = ma_5.ema_indicator()
    df['ma_20'] = ma_20.ema_indicator()
    df['ma_60'] = ma_60.ema_indicator()
    return df

def concat_trader_per_symbol(trader, symbol):
    """
    constraint:
    當日買賣成交金額 >= 1000w
    """
    df0 = loop.run_until_complete(do_find_trader_correlation(trader, symbol, B_endDate, B_startDate))
    df1 = loop.run_until_complete(do_find_symbol_correlation(symbol, S_endDate, S_startDate))
    df2 = loop.run_until_complete(do_find_margin_correlation(symbol, M_endDate, M_startDate))
    df3 = loop.run_until_complete(do_find_Institution_correlation(symbol, I_endDate, I_startDate))
    
    # add bband, MACD, RSI ....
    #df1 = wrap_ta_bband(df1)
    
    dfx = pd.concat([df0, df1, df2, df3],  axis=1, join="outer")
    dfp = dfx[dfx['C']*dfx['call'] >= 8000] # 找call成交價 >= 1000w(隔日沖) 才有效, 長線要把 filter 設小
    dfc = dfx[dfx['C']*dfx['put'] >= 8000]  # 找put 成交價 >= 1000w
    dfx = pd.concat([dfp, dfc], axis=0, join="outer").drop_duplicates()
    dfx['symbol'] = [symbol] * dfx.index.size
    dfx['tdname'] = [trader] * dfx.index.size
    return dfx

def concat_trader_profit_st0_df(df):
    """
    constraint: 
    1. 先買入, 再賣出
    2. 一定要有買入 cost, 再算賣出的 net profit
    """
    df['cost'] = np.where(df['call'] > df['put'], -(df['call'] - df['put'])*df['C'], 0)
    df['netf'] = np.where(df['put'] > df['call'],  (df['put'] - df['call'])*df['C'], 0)
    df['prft'] = [0] * df.index.size
    df['prft'][-1] = df['netf'].sum() + df['cost'].sum() if df['cost'].sum() < 0 else 0
    df = df.drop(columns=['cost', 'netf'])
    return df

def profile_trader_profit_st0_df(trader, limit=3):
    dfx = pd.DataFrame()
    
    df = loop.run_until_complete(do_find_trader_with_all_symbols()) 
    symbols = df['symbol'].tolist()
    
    for symbol in symbols:
        #print ("test tdname:{} - symbol:{}".format(trader, symbol))
        try:
            dfc = concat_trader_per_symbol(trader, symbol)
            dfc = concat_trader_profit_st0_df(dfc)
        except Exception as e:
            #print (e)
            continue

        dfx = dfx.append(dfc).drop_duplicates()
    
    dfx = dfx.sort_values(by='prft', ascending=False)
    return dfx[:limit], dfx[-limit:] # win, lose


In [16]:
C_MAX = 1000
C_MIN = 10
V_MAX = 1000
#symbol = '3008'

yy, mm, dd = date.today().isoformat().split("-")[0:3]
cdate = "{0}-{1}-{2}".format(yy, mm, dd)
#cdate = '2021-05-17'
today = str_to_datetime(cdate)
B_endDate = today
B_startDate = B_endDate - timedelta(days=70)
S_endDate = today
S_startDate = S_endDate - timedelta(days=70)
M_endDate = today
M_startDate = M_endDate - timedelta(days=10)
I_endDate = today
I_startDate = I_endDate - timedelta(days=10)

In [17]:
dfwins, dfloss = pd.DataFrame(), pd.DataFrame()

# 1.找全市場 rank 賣家比重
# 2.filter 賣家跟 symbol
# 3.找 賣家過去 D[t-70] 對 symbol 的買賣紀錄
# 4.profit 
df = loop.run_until_complete(do_find_max_puts_weight_symbol()) 
traders = df['tdname'].tolist()

p0 = IntProgress(max=len(traders))
p0.description = "[scan] {} rank win/lose broker".format(B_endDate)
display(p0)


# collect all traders to pool
for i, trader in enumerate(traders):
    try:
        p0.value = i
        
#        dfwin, dflos = profile_trader_profit_st0_df(trader)
    except Exception as e:
        continue
        
#    dfwins = dfwins.append(dfwin).drop_duplicates()
#    dfloss = dfloss.append(dflos).drop_duplicates() 

KeyboardInterrupt: 

In [ ]:
# rank bestest trader call with symbol at during test days
dfwins = dfwins.sort_values(by=['prft'], ascending=False)
dfwins[:50]
#dfr
#if not dfx.empty:
#dfx = dfx.reset_index().style.apply(ask_color, column=['put', 'call'], axis=1)

In [ ]:
# rank worstest trader with symbol at during test days
dfloss = dfloss.sort_values(by='prft', ascending=True)
dfloss[:10]

In [17]:
C_MAX = 1000
C_MIN = 3
V_MAX = 1000
#symbol = '3008'

TEST_MODE = False

yy, mm, dd = date.today().isoformat().split("-")[0:3]
cdate = "{0}-{1}-{2}".format(yy, mm, dd)
#cdate = '2021-02-26'
today = str_to_datetime(cdate)
B_endDate = today
B_startDate = B_endDate - timedelta(days=10)
S_endDate = today
S_startDate = S_endDate - timedelta(days=70)
M_endDate = today
M_startDate = M_endDate - timedelta(days=10) 
I_endDate = today
I_startDate = I_endDate - timedelta(days=10)


In [18]:
def scan_up_trend_strategy0(symbol):
    """
    constraint:
    ex: 
    [D0(t-1), D1(跳空漲), D2(t+1), D3(t+2)]
    [D0(t-2), D1(t-1),  D2(跳空漲), D3(t+1)]
    
    # 1.filter 跳空漲點到EndTime 每個bar的L不能低於跳空起漲點 
    # 2.跳空漲點在 inside [BBand.M:BBand.H]  
    # 3.確定跳空點的 T-n 的bar的L, 不能夠大於1.3倍的價差, 確保是波段起漲點
    # 4.[T-4] 到跳空點, 法人投信同步買超
    """
    df0 = loop.run_until_complete(do_find_symbol_correlation(symbol, S_endDate, S_startDate))
    df2 = loop.run_until_complete(do_find_margin_correlation(symbol, M_endDate, M_startDate))
    df3 = loop.run_until_complete(do_find_Institution_correlation(symbol, I_endDate, I_startDate))

    dfx = pd.concat([df0, df2, df3],  axis=1, join="outer")
    dfx = wrap_ta_bband(dfx)
    dfx['prec'] = dfx['C'].shift()
    dfx['diff'] = dfx['O'] / dfx['prec']
    
    if TEST_MODE:
        train, test = dfx.iloc[:-5], dfx.iloc[-5:]
        warmup, dfx = train.iloc[:-4], train.iloc[-4:]
        
    idx_last = np.where((dfx['diff'] >= 1.03) & (dfx['C'] >= dfx['O']))[0][-1]
    
    cnds = [
        (((dfx.iloc[idx_last:]['L'] > dfx.iloc[idx_last-1]['C']).sum() == dfx.iloc[idx_last:].index.size) and (dfx.iloc[idx_last:].index.size < 4)),
        (dfx.iloc[idx_last]['O'] >= dfx.iloc[idx_last]['bb_bbm']),
        ((dfx.iloc[idx_last-4:idx_last]['L']).min() * 1.3 > dfx.iloc[idx_last]['O']),
        (((dfx.iloc[idx_last-4:]['ForV'] > 0) & (dfx.iloc[idx_last-4:]['InvV'] > 0)).sum() >= 1)
    ]
 
    if sum([cnd for cnd in cnds]) == len(cnds):
        dfx = dfx.iloc[[idx_last]]
        dfx['symbol'] = [symbol]
        if TEST_MODE:
            dfx['Test_E'] = [test.iloc[-1]['C'] - dfx.iloc[-1]['C']]
        return dfx
    return pd.DataFrame()


In [19]:
def scan_up_trend_strategy1(symbol):
    """
    constraint:
    ex: 
    [D0(t-1), D1(), D2(t+1), D3(t+2)]
    [D0(t-2), D1(t-1),  D2(), D3(t+1)]
    
    # 1.filter symbol min(L[T+1], L[T+2]) >= (O[T])
    # 3.買進點 C[T]在 inside [BBand.:BBand.L]  
    # 4. 
    """
    df0 = loop.run_until_complete(do_find_symbol_correlation(symbol, S_endDate, S_startDate))
    df2 = loop.run_until_complete(do_find_margin_correlation(symbol, M_endDate, M_startDate))
    df3 = loop.run_until_complete(do_find_Institution_correlation(symbol, I_endDate, I_startDate))
    
    dfx = pd.concat([df0, df2, df3],  axis=1, join="outer")
    dfx = wrap_ta_bband(dfx)
    
    if TEST_MODE:
        train, test = dfx.iloc[:-5], dfx.iloc[-5:]
        dfx = train[:]
    
    idx_last = np.where((dfx['L'] * 1.05 < dfx['H']) & (dfx['O'] * 1.02 < dfx['C']))[0][-1]
    
    cnds = [
        (((dfx.iloc[idx_last+1:]['L'] > dfx.iloc[idx_last]['L']).sum() == dfx.iloc[idx_last+1:].index.size) and (dfx.iloc[idx_last+1:].index.size < 4)),
        (dfx.iloc[idx_last]['C'] >= dfx.iloc[idx_last]['bb_bbl']),
        (((dfx.iloc[idx_last-4:]['ForV'] > 0) & (dfx.iloc[idx_last-4:]['InvV'] > 0)).sum() > 1),
        ((dfx.iloc[idx_last-4:idx_last]['V']).mean() * 1.5 < dfx.iloc[idx_last]['V']),
    ]
    
    if sum([cnd for cnd in cnds]) == len(cnds):
        dfx = dfx.iloc[[idx_last]]
        dfx['symbol'] = [symbol]
        if TEST_MODE:
            dfx['Test_E'] = [test.iloc[-1]['C'] - dfx.iloc[-1]['C']]        
        return dfx
    return pd.DataFrame()


In [20]:
def scan_up_trend_strategy2(symbol):
    """
    constraint:
    ex: 
    [D0(t-1), D1(底部長紅支撐), D2(t+1), D3(t+2)]
    [D0(t-2), D1(t-1),  D2(底部長紅支撐), D3(t+1)]
    
    # 1.filter symbol min(L[T+1], L[T+2]) >= (O[T])
    # 3.買進點 C[T]在 inside [BBand.:BBand.L]  
    # 4. 
    """
    df0 = loop.run_until_complete(do_find_symbol_correlation(symbol, S_endDate, S_startDate))
    df2 = loop.run_until_complete(do_find_margin_correlation(symbol, M_endDate, M_startDate))
    df3 = loop.run_until_complete(do_find_Institution_correlation(symbol, I_endDate, I_startDate))
    
    dfx = pd.concat([df0, df2, df3],  axis=1, join="outer")
    dfx = wrap_ta_bband(dfx)
    dfx = wrap_ta_ema(dfx)
    
    if TEST_MODE:
        train, test = dfx.iloc[:-5], dfx.iloc[-5:]
        dfx = train[:]
    
    idx_last = np.where((dfx['L'] == dfx['L'].min()))[0][-1]
    
    cnds = [
        (((dfx.iloc[idx_last+1:]['L'] > dfx.iloc[idx_last]['L']).sum() == dfx.iloc[idx_last+1:].index.size) and (dfx.iloc[idx_last+1:].index.size < 4)),
        ((dfx.iloc[idx_last-4:idx_last]['V']).mean() *1.5 < dfx.iloc[idx_last]['V']),
        (((dfx.iloc[idx_last-4:]['ForV'] > 0) & (dfx.iloc[idx_last-4:]['InvV'] > 0)).sum() > 1),
        ((dfx.iloc[idx_last]['L'] * 1.05 < dfx.iloc[idx_last]['C']))
    ]
    
    if sum([cnd for cnd in cnds]) == len(cnds):
        dfx = dfx.iloc[[idx_last]]
        dfx['symbol'] = [symbol]
        if TEST_MODE:
            dfx['Test_E'] = [test.iloc[-1]['C'] - dfx.iloc[-1]['C']]    
        return dfx
    return pd.DataFrame()

In [21]:
def scan_up_trend_strategy3(symbol):
    """
    constraint:
    ex: 
    [D0(t-1), D1(跳空漲停), D2(跳空漲停t+1), D3(t+2)]
    [D0(t-2), D1(t-1),  D2(跳空漲停), D3(跳空漲停t+1)]
    
    # 1.filter symbol min(L[T+1], L[T+2]) >= (O[T])
    # 3.買進點 C[T]在 inside [BBand.:BBand.L]  
    # 4. 
    """
    df0 = loop.run_until_complete(do_find_symbol_correlation(symbol, S_endDate, S_startDate))
    df2 = loop.run_until_complete(do_find_margin_correlation(symbol, M_endDate, M_startDate))
    df3 = loop.run_until_complete(do_find_Institution_correlation(symbol, I_endDate, I_startDate))
    
    dfx = pd.concat([df0, df2, df3],  axis=1, join="outer")
    dfx = wrap_ta_bband(dfx)
    dfx = wrap_ta_ema(dfx)
    
    if TEST_MODE:
        train, test = dfx.iloc[:-5], dfx.iloc[-5:]
        dfx = train[:]
    
    dfx['prec'] = dfx['C'].shift()
    dfx['diff'] = (dfx['O'] - dfx['prec']) / dfx['prec'] * 100  
    
    idx_lasts = np.where((dfx['diff'] >= 9.5) & (dfx['C'] >= dfx['O']))[0]
    idx_last = idx_lasts[-1]
    
    cnds = [
        (((dfx.iloc[idx_last+1:]['L'] > dfx.iloc[idx_last]['L']).sum() == dfx.iloc[idx_last+1:].index.size) and (dfx.iloc[idx_last+1:].index.size < 4)),
        len(idx_lasts) >= 2
    ]
    
    if sum([cnd for cnd in cnds]) == len(cnds):
        dfx = dfx.iloc[[idx_last]]
        dfx['symbol'] = [symbol]
        if TEST_MODE:
            dfx['Test_E'] = [test.iloc[-1]['C'] - dfx.iloc[-1]['C']]    
        return dfx
    return pd.DataFrame()

In [22]:
def scan_up_trend_strategy4(symbol):
    """
    constraint:
    ex: 
    [D0(t-1), D1(爆量收紅), D2(爆量收紅t+1), D3(t+2)]
    [D0(t-2), D1(t-1),  D2(爆量收紅), D3(爆量收紅t+1)]
    
    # 4. 
    """
    df0 = loop.run_until_complete(do_find_symbol_correlation(symbol, S_endDate, S_startDate))
    df2 = loop.run_until_complete(do_find_margin_correlation(symbol, M_endDate, M_startDate))
    df3 = loop.run_until_complete(do_find_Institution_correlation(symbol, I_endDate, I_startDate))
    
    dfx = pd.concat([df0, df2, df3],  axis=1, join="outer")
    dfx = wrap_ta_bband(dfx)
    dfx = wrap_ta_ema(dfx)
    
    if TEST_MODE:
        train, test = dfx.iloc[:-5], dfx.iloc[-5:]
        dfx = train[:]
    
    dfx['prec'] = dfx['V'].describe()
    dfx['diff'] = (dfx['O'] - dfx['prec']) / dfx['prec'] * 100  
    
    idx_lasts = np.where((dfx['diff'] >= 9.5) & (dfx['C'] >= dfx['O']))[0]
    idx_last = idx_lasts[-1]
    
    cnds = [
        (((dfx.iloc[idx_last+1:]['L'] > dfx.iloc[idx_last]['L']).sum() == dfx.iloc[idx_last+1:].index.size) and (dfx.iloc[idx_last+1:].index.size < 4)),
        len(idx_lasts) >= 2
    ]
    
    if sum([cnd for cnd in cnds]) == len(cnds):
        dfx = dfx.iloc[[idx_last]]
        dfx['symbol'] = [symbol]
        if TEST_MODE:
            dfx['Test_E'] = [test.iloc[-1]['C'] - dfx.iloc[-1]['C']]    
        return dfx
    return pd.DataFrame()

In [23]:
symbols = abupy.ABuMarket.all_symbol()

if '5236' in symbols:
    print ("found")

In [24]:
from ipywidgets import IntProgress
from IPython.display import display

#symbols = ['2465']

p0 = IntProgress(max=len(symbols))
p0.description = "[scan] {} up-trend strategy0".format(S_endDate)
display(p0)

IntProgress(value=0, description='[scan] 2021-07-08 00:00:00 up-trend strategy0', max=2260)

In [25]:
# run trend strategy2
dfups = pd.DataFrame()
for i, symbol in enumerate(symbols):
    try:
        p0.value = i
        dfup = scan_up_trend_strategy0(symbol)
    except Exception as e:
        #print (e)
        continue
        
    dfups = dfups.append(dfup).drop_duplicates()

In [26]:
dfups = dfups.sort_values(by=['C'], ascending=False)
dfups[:20]

,O,H,L,C,V,ForV,InvV,DelV,bb_bbm,bb_bbh,bb_bbl,prec,diff,symbol
date,,,,,,,,,,,,,,
2021-07-07,175.50,178.0,171.5,175.5,5295.0,206.0,74.0,-142.0,153.1250,176.6130,129.6370,154.5,1.1359,6138
2021-07-06,123.50,129.0,120.0,124.5,71616.0,1900.0,0.0,-84.0,103.2900,117.2311,89.3489,112.0,1.1027,3260
2021-07-05,117.00,124.0,117.0,118.0,15541.0,-126.0,71.0,118.0,109.4000,120.5786,98.2214,112.0,1.0446,2375
2021-07-08,100.00,104.5,97.5,104.5,16357.0,1102.0,2.0,50.0,86.6150,97.9445,75.2855,95.0,1.0526,3588
2021-07-06,92.30,94.7,88.8,92.6,38671.0,1189.0,-1164.0,-72.0,68.2600,92.0143,44.5057,79.7,1.1581,2481
2021-07-06,75.40,79.5,74.2,78.8,4769.0,722.0,70.0,79.0,72.6200,75.9902,69.2498,72.4,1.0414,8942
2021-07-06,32.55,32.8,32.4,32.7,11604.0,1586.0,268.0,-174.0,30.7325,31.8471,29.6179,31.0,1.0500,3231


In [27]:
# run trend strategy2
dfups = pd.DataFrame()
for i, symbol in enumerate(symbols):
    try:
        p0.value = i
        dfup = scan_up_trend_strategy3(symbol)
    except Exception as e:
        #print (e)
        continue
        
    dfups = dfups.append(dfup).drop_duplicates()

In [28]:
dfups = dfups.sort_values(by=['C'], ascending=False)
dfups[:20]

,O,H,L,C,V,ForV,InvV,DelV,bb_bbm,bb_bbh,bb_bbl,ma_5,ma_20,ma_60,prec,diff,symbol
date,,,,,,,,,,,,,,,,,
2021-07-06,91.80,91.80,91.80,91.80,250.0,NaN,NaN,NaN,NaN,NaN,NaN,86.0568,82.6469,NaN,83.50,9.9401,4152
2021-07-08,21.55,21.55,21.55,21.55,4906.0,-59.0,0.0,0.0,NaN,NaN,NaN,20.0235,20.0231,NaN,19.60,9.9490,4157
2021-07-08,12.70,12.70,12.70,12.70,701.0,0.0,0.0,0.0,7.9815,10.6147,5.3483,10.1978,8.4165,NaN,10.50,20.9524,6287
2021-07-07,7.45,7.45,7.45,7.45,258.0,NaN,NaN,NaN,NaN,NaN,NaN,6.7035,6.3147,NaN,6.78,9.8820,3191
2021-07-06,6.90,7.00,6.79,7.00,49.0,0.0,0.0,0.0,NaN,NaN,NaN,6.3192,6.1150,NaN,5.90,16.9492,4304
2021-07-06,4.10,4.11,4.10,4.11,253.0,0.0,0.0,0.0,NaN,NaN,NaN,3.7157,3.6339,NaN,3.74,9.6257,6225
2021-07-08,3.99,3.99,3.99,3.99,3648.0,0.0,0.0,0.0,NaN,NaN,NaN,3.4742,3.0501,NaN,3.63,9.9174,5701
2021-07-08,2.55,2.55,2.55,2.55,2832.0,3.0,0.0,0.0,1.8245,2.2799,1.3691,2.2095,1.9101,NaN,2.32,9.9138,911616
